In [1]:
import polars as pl
import pandas as pd

## Abro sitios y reviews

In [2]:
pl_sitios = pl.read_parquet('parquet/sitios_FL.parquet')
pl_reviews = pl.read_parquet('parquet/review_florida.parquet')
pl_reviews.shape

(2850000, 9)

## Creo un dataframe a partir de un diccionario que est compuesto por las combinaciones del
## nombre del restaurant + relative_results
## Se llama pl_name_review

In [3]:
i = 0
cantidad = pl_sitios.shape[0]
dicc = {'name':[],'gmap_id':[]}
for i in range(0,cantidad-1):
    if pl_sitios[i,'relative_results']:
        lista = pl_sitios[i,'relative_results'].split(',')
        for elem in lista:
            dicc['name'].append(pl_sitios[i,'name'])
            dicc['gmap_id'].append(elem)
    i = i + 1
pl_review_FL = pl.DataFrame(dicc)    
pl_review_FL.shape

(13788, 2)

## Hago un join entre 
## pl_name_review y pl_reviews

In [4]:
pl_review_FL = pl_review_FL.join(pl_reviews.select(['gmap_id','name','rating','text']), on='gmap_id')

In [5]:
pl_review_FL = (pl_review_FL.groupby(['name','name_right']).agg([
    pl.col('rating').mean(),
    pl.col('text').max()
])
 .sort('name')
)

## Elimino duplicados

pl_review_FL = pl_review_FL.groupby(['name','gmap_id']).agg([
    pl.col('rating').mean(),
    pl.col('text')
]).sort('name')
pl_review_FL = pl_review_FL.with_columns([
    pl.col('rating').apply(lambda x: round(x,2))
])
pl_review_FL.shape

In [6]:
pl_review_FL.columns

['name', 'name_right', 'rating', 'text']

In [7]:
pl_review_FL.shape

(247328, 4)

## Salvo como rest_review_FL(parquet y csv)

In [8]:
# No puedo salvar csv por la lista y me dio guerra de momento convertirla en string. Lo dejo para mas adelante
pl_review_FL.write_parquet('parquet/review_FL.parquet')
pl_review_FL.write_csv('csv/review_FL.csv')
pl_review_FL.write_parquet('PG_Google_Maps/Ornaldo/review_FL.parquet')
pl_review_FL.write_csv('PG_Google_Maps/Ornaldo/review_FL.csv')